In [1]:
%matplotlib qt
#%matplotlib inline

import os
import re
import matplotlib as plt


os.chdir('/Users/zach/Documents/git/nba_bets/src/models/')

from train_model_functions import *


In [2]:
data_location = '/Users/zach/Documents/git/nba_bets/data/'
in_delim = '|'

target = 'WIN_HOME'

# specify directories 
data_dir = Path(data_location)
processed_dir = data_dir / 'processed'
raw_dir = data_dir / 'raw'

In [10]:
# specify predictors
predictors = [
'TEAM_FEATURE_cumulative_pt_pct_COURT_HOME_AWAY_diff',
'TEAM_FEATURE_cumulative_win_pct_COURT_HOME_AWAY_diff',
 'TEAM_FEATURE_cumulative_count_GAME_NUMBER_HOME',
'TEAM_FEATURE_pct_pct_prev_COURT_HOME_AWAY_diff',
#'TEAM_FEATURE_cumulative_pt_pct_COURT_HOME'
]

In [4]:
# parameter dict -- defines grid of parameter combinations to test
model_param_dict = {
    'learning_rate':[0.001],
    'n_estimators':[750],
    #'max_depth':[3,5,10],
    'min_samples_leaf':[1],
    'min_child_weight':[1],
    'gamma':[0.5],
    'subsample':[1],
    'colsample_bytree':[1.0],
    'objective':['binary:logistic'],
    'nthread':[1],
    'scale_pos_weight':[1],
    'seed':[223],
    'train_frac':[0.5],
    'bet_dist_type':['max-ep', 'opt'],
    'bet_budget':[100],
    'min_exp_roi':[0.1],
    #'conf_threshold':[0.2],

    # functions
     'training_prediction_functions':[(train_rf_classifier, predict_with_rf_classifier)],
    #'training_prediction_functions':[(train_logistic_cv_classifier, predict_with_logistic_classifier)],
    'model_performance_function':[calc_model_performance],
    'bet_recommendation_function':[calc_bet_distribution],
    'bet_performance_function':[calc_total_profit],
    'modeling_round_var':[['SEASON_NUMBER']],
    #'modeling_round_var':[['GAME_DATE_year','GAME_DATE_week_number']],
    'betting_round_params':[(['GAME_DATE_year','GAME_DATE_week_number'], [2016, 46], [2017, 15])]
    ,'penalty':['elasticnet']
    ,'max_iter':[10000]
}

# list of parameter combinations
model_params = make_parameter_dict_list(model_param_dict)

In [5]:
##################
# read in data
##################
train_data_init = pd.read_csv(processed_dir / 'training_data.csv', in_delim)



In [11]:
##################
# run model training/testing simulation
##################
simulator = Simulator(param_dict_list=model_params, training_data=train_data_init, predictors=predictors, target=target)

simulator.simulate()




In [12]:
sim_out = simulator.simulation_output()
agg_sim_out = simulator.aggregate_simulation_output()

In [13]:
sim_out

,bet_budget,bet_dist_type,bet_performance_function,bet_recommendation_function,betting_round_params,colsample_bytree,gamma,learning_rate,max_iter,min_child_weight,...,min_profit,sharpe_ratio_profit,total_exp_roi,roi,auc,logloss,accuracy,cumulative_profit,param_num_agg,param_num
0,100,max-ep,calc_total_profit,calc_bet_distribution,"(['GAME_DATE_year', 'GAME_DATE_week_number'], ...",1.0,0.5,0.001,10000,1,...,-0.000000,0.540033,2.629093,9.160000,0.477647,0.718671,0.523810,916.000000,1.0,1.0
1,100,max-ep,calc_total_profit,calc_bet_distribution,"(['GAME_DATE_year', 'GAME_DATE_week_number'], ...",1.0,0.5,0.001,10000,1,...,-100.000000,0.631029,4.574009,-1.000000,0.709478,0.638518,0.574074,816.000000,1.0,1.0
2,100,max-ep,calc_total_profit,calc_bet_distribution,"(['GAME_DATE_year', 'GAME_DATE_week_number'], ...",1.0,0.5,0.001,10000,1,...,-100.000000,0.714267,3.557312,-1.000000,0.565217,0.726859,0.540000,716.000000,1.0,1.0
3,100,max-ep,calc_total_profit,calc_bet_distribution,"(['GAME_DATE_year', 'GAME_DATE_week_number'], ...",1.0,0.5,0.001,10000,1,...,-0.000000,0.915402,4.084422,8.000000,0.782099,0.629914,0.561404,1516.000000,1.0,1.0
4,100,max-ep,calc_total_profit,calc_bet_distribution,"(['GAME_DATE_year', 'GAME_DATE_week_number'], ...",1.0,0.5,0.001,10000,1,...,-100.000000,0.440908,2.987276,-1.000000,0.735294,0.580562,0.673077,1416.000000,1.0,1.0
5,100,max-ep,calc_total_profit,calc_bet_distribution,"(['GAME_DATE_year', 'GAME_DATE_week_number'], ...",1.0,0.5,0.001,10000,1,...,-100.000000,0.396874,2.814664,-1.000000,0.640870,0.673497,0.562500,1316.000000,1.0,1.0
6,100,max-ep,calc_total_profit,calc_bet_distribution,"(['GAME_DATE_year', 'GAME_DATE_week_number'], ...",1.0,0.5,0.001,10000,1,...,-100.000000,0.404271,3.623631,-1.000000,0.705405,0.581537,0.711538,1216.000000,1.0,1.0
7,100,max-ep,calc_total_profit,calc_bet_distribution,"(['GAME_DATE_year', 'GAME_DATE_week_number'], ...",1.0,0.5,0.001,10000,1,...,-100.000000,0.387281,2.442954,-1.000000,0.729839,0.615163,0.672727,1116.000000,1.0,1.0
8,100,max-ep,calc_total_profit,calc_bet_distribution,"(['GAME_DATE_year', 'GAME_DATE_week_number'], ...",1.0,0.5,0.001,10000,1,...,-100.000000,0.541938,3.528594,-1.000000,0.604444,0.635332,0.577778,1016.000000,1.0,1.0
9,100,max-ep,calc_total_profit,calc_bet_distribution,"(['GAME_DATE_year', 'GAME_DATE_week_number'], ...",1.0,0.5,0.001,10000,1,...,-100.000000,0.570873,3.024743,-1.000000,0.589314,0.657837,0.596154,916.000000,1.0,1.0


In [17]:
sim_out.sort_values(['GAME_DATE_year','GAME_DATE_week_number'])
sim_out['year_week'] = [str(s1)+'-'+str(s2) for s1, s2 in zip(sim_out['GAME_DATE_year'],sim_out['GAME_DATE_week_number'])]
sim_out['cumulative_bet_amount'] = sim_out[['param_num','total_bet_amount']].groupby(['param_num']).cumsum()
sim_out['cumulative_roi'] = sim_out['cumulative_profit'] / sim_out['cumulative_bet_amount']
sim1 = sim_out[sim_out['param_num'] == 1]

x = sim1['year_week']# np.linspace(1,sim1.shape[0],sim1.shape[0])
fig, ax = plt.subplots(1,1)
#ax[0].set(title="Cumulative profit by week", xlabel="Year-Week", ylabel="Cumulative Profit")
#ax[0].set_xticklabels(sim1['year_week'],rotation=45,ha='right')

ax.set(title="Cumulative ROI by week", xlabel="Year-Week", ylabel="Cumulative ROI")
ax.set_xticklabels(sim1['year_week'],rotation=45,ha='right')
ax.set_ylim([0,2])
# ax[1].set(title="Log loss by week", xlabel="Year-Week", ylabel="Log Loss")
# ax[1].set_xticklabels(sim1['year_week'],rotation=45,ha='right')

for k in sim_out['param_num'].unique():
    #y0 = sim_out[sim_out['param_num'] == k]['cumulative_profit']
    #ax[0].plot(x,y0,label='param num '+str(k))
    y1 = sim_out[sim_out['param_num'] == k]['cumulative_roi']
    ax.plot(x,y1,label='param num '+str(k))
   # y2 = sim_out[sim_out['param_num'] == k]['logloss']
   # ax[1].plot(x,y2,label='param num '+str(k))

plt.tight_layout()
plt.legend()
plt.show()